In [17]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import numpy as np
from itertools import islice
from numpy import random

In [18]:
df = pd.read_csv('cleaned/dataframe')
df.set_index('Unnamed: 0', inplace=True)
df.index.names = ['date']
df.head()

,TSLA,MSFT,GOOG,PFE,AAPL,AMZN,SPY,ICLN,XOM,WBA
date,,,,,,,,,,
2010-06-29,4.778,18.237944,226.281845,8.999600,7.867191,108.610001,83.951881,11.147953,37.713707,20.105280
2010-06-30,4.766,18.003220,221.644226,8.986997,7.724694,109.260002,83.154335,11.109907,37.568890,20.264664
2010-07-01,4.392,18.120586,218.924423,8.968088,7.631025,110.959999,82.783760,11.330582,37.266071,20.158405
2010-07-02,3.840,18.206648,217.459900,8.911369,7.583728,109.139999,82.332611,11.528431,37.239731,20.006615
2010-07-06,3.222,18.636972,217.220810,9.005903,7.635632,110.059998,82.872383,11.680624,37.825611,20.203953


In [19]:
# 选出最后5%数据作为测试数据
dates = sorted(df.index.values)
last_5pct = sorted(df.index.values)[-int(0.05*len(dates))]
df_val = df[(df.index >= last_5pct)]
df_train = df[(df.index < last_5pct)]
df_train

,TSLA,MSFT,GOOG,PFE,AAPL,AMZN,SPY,ICLN,XOM,WBA
date,,,,,,,,,,
2010-06-29,4.778000,18.237944,226.281845,8.999600,7.867191,108.610001,83.951881,11.147953,37.713707,20.105280
2010-06-30,4.766000,18.003220,221.644226,8.986997,7.724694,109.260002,83.154335,11.109907,37.568890,20.264664
2010-07-01,4.392000,18.120586,218.924423,8.968088,7.631025,110.959999,82.783760,11.330582,37.266071,20.158405
2010-07-02,3.840000,18.206648,217.459900,8.911369,7.583728,109.139999,82.332611,11.528431,37.239731,20.006615
2010-07-06,3.222000,18.636972,217.220810,9.005903,7.635632,110.059998,82.872383,11.680624,37.825611,20.203953
...,...,...,...,...,...,...,...,...,...,...
2019-12-24,85.050003,155.014450,1343.560059,35.082932,70.234581,1789.209961,313.286652,11.603066,62.843296,54.881516
2019-12-26,86.188004,156.285065,1360.400024,35.163296,71.628067,1868.770020,314.954376,11.672252,62.942028,55.398830
2019-12-27,86.075996,156.570709,1351.890015,35.109722,71.600876,1869.800049,314.876373,11.632717,62.726620,55.511688


In [20]:
b = 32                                      # 说真的我真不知道这个b是啥用
f = 8                                       # 说真的我真不知道这个f是啥用
w = b + f
sequential_data = []
prev_days = deque(maxlen=w)                 # deque: 头尾皆可操作的queue 此处"前一天"导入40个
for col in df_train.columns:
    df = df_train[col].pct_change()         # df.pct_change(): 表示当前元素与先前元素的相差百分比，当然指定periods=n,表示当前元素与先前n 个元素的相差百分比
    df.dropna(inplace=True)                 # df.dropna(): 用于滤除缺失数据
    # print(df)
    seq = []
    for i in df.values:
        prev_days.append(i)
        if len(prev_days) == w:             # 够40个就存一个seq
            pd = np.array(list(prev_days))  # list array化
            b_seq = pd[:b]                  # pd前b个 -> b序列
            b_min_max = max((abs(b_seq.min()), abs(b_seq.max()))) # 离0最远的
            b_seq = np.interp(b_seq, (-b_min_max, b_min_max), (-1, +1)) # 算出对应b_seq的y轴值(32)
            f_seq = np.interp(pd[-f:], (-b_min_max, b_min_max), (-1, +1)) # 算出对应f_seq的y轴值(8)
            seq.append([np.array(b_seq), np.array(f_seq)])  # append 40 个 应变量
    sequential_data.append(seq)             # append 区块
for i in range(len(sequential_data)):
    random.shuffle(sequential_data[i])      # 将序列的所有元素随机排序
sequential_data = np.array(sequential_data) # array化
np.save('preprocessed/prices_32_8', sequential_data)

<ipython-input-20-824baf973c73>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sequential_data = np.array(sequential_data) # array化


In [21]:
print(len(sequential_data)) # Assets
print(len(sequential_data[1])) # Sequences
print(len(sequential_data[2][2])) # b, f
print(len(sequential_data[3][3][0])) # b
print(len(sequential_data[3][3][1])) # f
print(sequential_data[3][3][0])
print(sequential_data[3][3][1])

10
2393
2
32
8
[ 0.74182832 -0.37146853  0.65854951  0.20369607  0.41831303  0.69090696
  0.03713173 -0.16077868 -0.45918467 -0.1002698  -0.41456477 -0.15209693
  0.16532519  0.22808832 -0.56744522 -0.29363626 -0.14132581 -0.15464448
  0.56894536 -0.37043476 -0.643839   -0.953266    1.          0.62644121
 -0.12876558 -0.51651095  0.33952758  0.12963145  0.41365829  0.24344503
  0.25490618 -0.20280319]
[ 0.15278304  0.          0.         -0.03807415  0.43179849 -0.47814518
  0.48313824  0.64171144]


In [23]:
np.save('./preprocessed/prices', sequential_data)